# Preparação do Ambiente

## Instalação dos pacotes necessários

In [ ]:
!pip install nltk gensim spacy sentence-transformers scikit-learn seaborn matplotlib pandas hdbscan
!python -m spacy download pt_core_news_sm

## Importação das bibliotecas
Após a instalação, tem que reiniciar a sessão.

In [ ]:
import os
import re
import time
import zipfile
from collections import Counter

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from IPython.display import display

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy

from gensim.models import FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from sentence_transformers import SentenceTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from scipy.spatial.distance import cdist
from hdbscan import HDBSCAN

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# Definição de Funções Úteis

## Carregar corpus

In [ ]:
def carregar_corpus(path, qtde_max = 999999999):
    ids = []
    ementas = []

    arquivos = [f for f in os.listdir(path) if f.endswith('.txt')]

    cont = 0
    for arquivo in arquivos:
        if cont == qtde_max:
            break
        cont += 1

        id_acordao = os.path.splitext(arquivo)[0]  # remove .txt
        with open(os.path.join(path, arquivo), 'r', encoding='utf-8') as f:
            texto = f.read().strip()

        # Remove a citação final entre parênteses, se houver
        texto_limpo = re.sub(r'\s*\(TRT.*\)$', '', texto)

        ids.append(id_acordao)
        ementas.append(texto_limpo)

    df = pd.DataFrame({'id_acordao': ids, 'ementa': ementas})
    df['ementa'] = df['ementa'].astype(str)

    print(f"Total de ementas carregadas: {len(df)}")
    print(df.sample(5))

    return df

## Contar palavras

In [ ]:
def contar_palavras(texto):
    if isinstance(texto, list):
        return len(texto)
    elif isinstance(texto, str):
        return len(texto.split())
    else:
        return 0



## Contar caracteres

In [ ]:
def contar_caracteres(texto):
    if isinstance(texto, list):
        return sum(len(p) for p in texto)
    elif isinstance(texto, str):
        return len(texto)
    else:
        return 0


## Lematizar texto

In [ ]:
def lematizar_texto(texto):
    doc = nlp_spacy(texto)
    lemas = [
        token.lemma_
        for token in doc
        if token.is_alpha and not token.is_stop
    ]
    return lemas


## Remover stopwords

In [ ]:
def remover_stopwords(text, stopwords_remover):

    if isinstance(text, list):
        text = ' '.join(text)  # transforma lista em string se for o caso

    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\d+", " ", text)
    tokens = word_tokenize(text.lower(), language="portuguese")

    tokens = [
        token
        for token in tokens
        if token not in stopwords_remover and len(token) > 2 and token.isalpha()
    ]

    return tokens

## Carregar palavras mais frequentes

In [ ]:
def carregar_palavras_mais_frequentes(lista_ementa, stopwords_ignorar, top_n=20):

    # Junta todas as ementas em um único texto
    texto_unificado = ' '.join([str(e) for e in lista_ementa])

    # Limpeza básica
    texto_unificado = re.sub(r"[^\w\s]", " ", texto_unificado)  # remove pontuação
    texto_unificado = re.sub(r"\d+", " ", texto_unificado)      # remove números
    texto_unificado = texto_unificado.lower()                   # minúsculas

    # Tokenização
    tokens = word_tokenize(texto_unificado, language='portuguese')

    # Remove stopwords e tokens não alfabéticos
    tokens_filtrados = [
        token for token in tokens
        if token not in stopwords_ignorar and len(token) > 2 and token.isalpha()
    ]

    # Contagem das palavras
    counter = Counter(tokens_filtrados)
    most_common = counter.most_common(top_n)

    # Separa palavras e frequências
    palavras, contagens = zip(*most_common) if most_common else ([], [])

    return palavras, contagens


## Gerar Embeddings

In [ ]:
def gerar_embeddings(corpus, modelo='sbert'):

    corpus_texto = [' '.join(doc) if isinstance(doc, list) else doc for doc in corpus]

    embeddings = []

    if modelo == 'fasttext':
        tokenizado = [doc.split() for doc in corpus_texto]
        ft_model = FastText(sentences=tokenizado, vector_size=300, window=5, min_count=2, workers=4, epochs=20)
        for doc in tokenizado:
            vetor = np.mean(
                [ft_model.wv[word] for word in doc if word in ft_model.wv] or [np.zeros(300)],
                axis=0
            )
            embeddings.append(vetor)

    elif modelo == 'doc2vec':
        tokenizado = [doc.split() for doc in corpus_texto]
        documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(tokenizado)]
        d2v_model = Doc2Vec(documents, vector_size=300, window=5, min_count=2, workers=4, epochs=40)
        for doc in tokenizado:
            if len(doc) == 0:
                vetor = np.zeros(300)
            else:
                vetor = d2v_model.infer_vector(doc)
            embeddings.append(vetor)

    elif modelo == 'sbert':
        sbert = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
        embeddings = sbert.encode(corpus_texto, show_progress_bar=True)

    elif modelo == 'bertimbau':
        sbert = SentenceTransformer('neuralmind/bert-base-portuguese-cased')
        embeddings = sbert.encode(corpus_texto, show_progress_bar=True)

    elif modelo == 'legalbert':
        sbert = SentenceTransformer('raquelsilveira/legalbertpt_fp')
        embeddings = sbert.encode(corpus_texto, show_progress_bar=True)

    else:
        raise ValueError("Modelo não reconhecido. Escolha entre 'fasttext', 'doc2vec', 'sbert', 'bertimbau' ou 'legalbert'.")

    embeddings = np.vstack(embeddings) if isinstance(embeddings, list) else np.array(embeddings)
    return embeddings


## Reduzir dimensionalidade

In [ ]:
def reduzir_dimensionalidade(embeddings, n_components=2):
    reducer = umap.UMAP(n_components=n_components, random_state=SEED)
    reduzido = reducer.fit_transform(embeddings)
    return reduzido


## Aplicar clusterização

In [ ]:
def aplicar_clusterizacao(embeddings, min_cluster_size=5):
    modelo = HDBSCAN(
        min_cluster_size=min_cluster_size,
        metric='euclidean',
        cluster_selection_method='eom'
    ).fit(embeddings)
    labels = modelo.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    print(f"🔸 HDBSCAN encontrou {n_clusters} clusters (sem contar ruído)")

    return labels, modelo


## Extrair termos relevantes cluster (tf-idf)

In [ ]:
def termos_por_cluster(textos, labels, top_n=10):

    textos_str = [' '.join(doc) if isinstance(doc, list) else doc for doc in textos]

    df = pd.DataFrame({'texto': textos_str, 'cluster': labels})

    resultados = {}
    for cluster in sorted(df['cluster'].unique()):
        docs = df[df['cluster'] == cluster]['texto']
        vectorizer = TfidfVectorizer(ngram_range=(1, 2))
        X = vectorizer.fit_transform(docs)
        indices = np.argsort(X.toarray().sum(axis=0))[::-1]
        termos = [vectorizer.get_feature_names_out()[i] for i in indices[:top_n]]
        resultados[cluster] = termos

    return resultados


## Avaliar clusters

In [ ]:
def avaliar_clusters(embeddings, labels, modelo):
    mask = labels != -1  # Ignorar ruído para HDBSCAN

    if len(set(labels[mask])) <= 1:
        print("⚠️ Apenas um cluster detectado (ou ruído excessivo). Avaliação não aplicável.")
        return {
            'Silhueta': np.nan,
            'Calinski-Harabasz': np.nan
        }

    resultados = {}

    resultados['Silhueta'] = silhouette_score(embeddings[mask], labels[mask])
    resultados['Calinski-Harabasz'] = calinski_harabasz_score(embeddings[mask], labels[mask])

    return resultados


## Calcular coherence score (c_v)

In [ ]:
def calcular_coherence(textos, labels, top_n=10):

    df = pd.DataFrame({'texto': textos, 'cluster': labels})
    df = df[df['cluster'] != -1].reset_index(drop=True)  # Remove ruídos e reseta os índices

    # Vetorizar todos os documentos
    docs_str = df['texto'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    tfidf = vectorizer.fit_transform(docs_str)
    termos = vectorizer.get_feature_names_out()

    topics = []

    for cluster in sorted(df['cluster'].unique()):
        docs_cluster = df[df['cluster'] == cluster]
        indices = docs_cluster.index
        submatriz = tfidf[indices]

        if submatriz.shape[0] == 0:
            continue

        # Soma TF-IDF dos termos no cluster e seleciona os top_n
        tfidf_soma = np.asarray(submatriz.sum(axis=0)).flatten()
        top_indices = tfidf_soma.argsort()[-top_n:][::-1]
        top_termos = [termos[i] for i in top_indices]
        topics.append(top_termos)

    # Preparar corpus e dictionary para CoherenceModel
    all_tokens = [doc.split() if isinstance(doc, str) else doc for doc in textos]
    dictionary = Dictionary(all_tokens)

    coherence_model = CoherenceModel(
        topics=topics,
        texts=all_tokens,
        dictionary=dictionary,
        coherence='c_v'
    )

    coherence = coherence_model.get_coherence()
    print(f"🔸 Coherence Score (TF-IDF): {coherence:.4f}")
    return coherence


## Plotar clusters

In [ ]:
def plotar_clusters(embeddings_reduzidos, labels, titulo='Clusters'):

    plt.figure(figsize=(10,7))
    palette = sns.color_palette("tab10", len(set(labels)))

    sns.scatterplot(
        x=embeddings_reduzidos[:, 0],
        y=embeddings_reduzidos[:, 1],
        hue=labels,
        palette=palette,
        legend='full',
        alpha=0.7,
        s=60,
        edgecolor='black'
    )

    plt.title(titulo, fontsize=14)
    plt.xlabel('Componente 1')
    plt.ylabel('Componente 2')
    plt.grid(visible=True, linestyle='--', linewidth=0.5, alpha=0.7)
    plt.legend(title='Cluster', loc='best')
    plt.tight_layout()
    plt.show()



## Pipeline avaliação embedding

In [ ]:
def pipeline_embedding(
    nome_modelo, textos, n_components=2,
    min_cluster_size=5
):

    print(f'\n🔹 Processando modelo: {nome_modelo}')

    inicio = time.time()
    embeddings = gerar_embeddings(textos, modelo=nome_modelo)

    reduzido = reduzir_dimensionalidade(embeddings, n_components=n_components)

    melhor_k = None

    labels, modelo_cluster = aplicar_clusterizacao(
        embeddings, min_cluster_size=min_cluster_size
    )

    tempo_embedding = time.time() - inicio
    print(f'⏳ Tempo Pipeline ({nome_modelo}): {tempo_embedding:.2f}s')

    avaliacao = avaliar_clusters(embeddings, labels, modelo_cluster)
    avaliacao['Modelo'] = nome_modelo
    avaliacao['K'] = (len(set(labels)) - (1 if -1 in labels else 0))
    avaliacao['Tempo Pipeline (s)'] = tempo_embedding
    avaliacao['Coherence c_v'] = calcular_coherence(textos, labels)

    return avaliacao, labels, embeddings, reduzido, avaliacao['K'], modelo_cluster


## Comparar modelos e coletar avaliações

In [ ]:
def comparar_modelos(
    modelos, textos, n_components=2,
    min_cluster_size=5
):
    resultados = []
    embeddings_dict = {}
    labels_dict = {}
    reduzidos_dict = {}
    k_dict = {}
    modelos_cluster = {}

    for modelo in modelos:
        avaliacao, labels, embeddings, reduzido, k, modelo_cluster = pipeline_embedding(
            modelo, textos, min_cluster_size=min_cluster_size,
            n_components=n_components
        )
        resultados.append(avaliacao)
        embeddings_dict[modelo] = embeddings
        labels_dict[modelo] = labels
        reduzidos_dict[modelo] = reduzido
        k_dict[modelo] = k
        modelos_cluster[modelo] = modelo_cluster

    df_resultados = pd.DataFrame(resultados)
    df_resultados.set_index('Modelo', inplace=True)

    return df_resultados, embeddings_dict, labels_dict, reduzidos_dict, k_dict, modelos_cluster


## Visualizar termos do cluster

In [ ]:
def visualizar_clusters_termos(textos, labels, modelo_nome, top_n=10, n_amostra=5):
    df = pd.DataFrame({'texto': textos, 'cluster': labels})
    clusters_unicos = [c for c in set(labels) if c != -1]
    clusters_amostrados = np.random.choice(clusters_unicos, min(n_amostra, len(clusters_unicos)), replace=False)

    print(f'\n🟦 Modelo: {modelo_nome}')
    print(f'Clusters amostrados: {clusters_amostrados}')

    for cluster in clusters_amostrados:
        docs = df[df['cluster'] == cluster]['texto']
        docs_tokens = [doc if isinstance(doc, list) else doc.split() for doc in docs]
        all_tokens = [token for doc in docs_tokens for token in doc]
        contador = Counter(all_tokens)
        termos_top = [t for t, _ in contador.most_common(top_n)]

        print(f'\n🔹 Cluster {cluster}:')
        print(f'Termos mais representativos: {termos_top}')
        print(f'Número de documentos no cluster: {len(docs)}')


# Execução

## Define variáveis e parâmetros gerais

In [ ]:
# Sementes globais
SEED = 42
np.random.seed(SEED)

QTDE_ACORDAOS_PROCESSADOS = 10000

# palavras mais comuns
TOP_N_WORDS = 55

# Caminho do arquivo
path = '/content/ementas'
df = carregar_corpus(path, QTDE_ACORDAOS_PROCESSADOS)

# Stopwords
stopwords_pt = set(stopwords.words("portuguese"))

# Carrega o modelo de português
nlp_spacy = spacy.load('pt_core_news_sm')


## 3. Análise Exploratória Antes do Pré-Processamento

### Amostra do corpus

In [ ]:
# Adiciona colunas de contagem
df['qtde_palavras'] = df['ementa'].apply(contar_palavras)
df['qtde_caractere'] = df['ementa'].apply(lambda x: len(str(x)))

# Gera amostra com 5 ementas
amostra = df[['id_acordao', 'ementa', 'qtde_palavras', 'qtde_caractere']].sample(5)

display(amostra)

### Nuvem de palavras (sem pré-processamento)

In [ ]:
text_all = ' '.join(df['ementa'])

wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(text_all)

plt.figure(figsize=(15,7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nuvem de Palavras (Antes do Pré-processamento)')
plt.show()


### Tamanho dos textos (em caracteres e palavras)

In [ ]:
df['n_caracteres'] = df['ementa'].apply(len)
df['n_palavras'] = df['ementa'].apply(contar_palavras)

print(df[['n_caracteres', 'n_palavras']].describe())


### Histograma do tamanho dos textos

In [ ]:
sns.set(style="whitegrid")

# Calcula estatísticas
media = df['n_palavras'].mean()
mediana = df['n_palavras'].median()
desvio = df['n_palavras'].std()

plt.figure(figsize=(14,7))
sns.histplot(df['n_palavras'], bins=50, kde=True, color='royalblue')

# Linhas de média e mediana
plt.axvline(media, color='red', linestyle='--', linewidth=2, label=f'Média: {media:.0f}')
plt.axvline(mediana, color='green', linestyle='-', linewidth=2, label=f'Mediana: {mediana:.0f}')

plt.title('Distribuição do Número de Palavras nas Ementas\n(Antes do Pré-processamento)', fontsize=16)
plt.xlabel('Número de Palavras por Ementa', fontsize=12)
plt.ylabel('Frequência de Ementas', fontsize=12)

plt.grid(visible=True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.legend()

### Frequência das Palavras Mais Comuns (stopwords customizadas)

In [ ]:
df['ementa_lematizada'] = df['ementa'].apply(lematizar_texto)

palavras, contagens = carregar_palavras_mais_frequentes(df['ementa_lematizada'], stopwords_pt, top_n = TOP_N_WORDS)
plt.figure(figsize=(12,12))
sns.barplot(x=list(contagens), y=list(palavras), palette='viridis')
plt.title(f'Top {TOP_N_WORDS} Palavras Mais Frequentes (Após Pré-processamento)')
plt.xlabel('Frequência')
plt.ylabel('Palavras')
plt.show()


In [ ]:
def analisar_distribuicao_palavras(contador):
    contagens_ordenadas = np.array(sorted(contador.values(), reverse=True))
    soma_total = contagens_ordenadas.sum()
    cumulativa = np.cumsum(contagens_ordenadas) / soma_total

    pontos = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
    resultados = {}

    for p in pontos:
        idx = np.searchsorted(cumulativa, p)
        resultados[p] = idx + 1  # +1 porque index começa no zero

    print("Palavras necessárias para atingir os percentuais acumulados:")
    for p, n in resultados.items():
        print(f"{int(p*100)}% -> {n} palavras")

    # Plotando a curva
    plt.figure(figsize=(12,6))
    plt.plot(range(1, len(cumulativa)+1), cumulativa, marker='.')
    plt.title('Curva de Frequência Cumulativa')
    plt.xlabel('Top N palavras')
    plt.ylabel('Frequência acumulada')
    plt.grid(True)

    # Marcar os pontos de interesse
    for p, n in resultados.items():
        plt.axvline(x=n, linestyle='--', color='red')
        plt.text(n, cumulativa[n-1], f'{int(p*100)}%', color='red')

    plt.show()

    return resultados

resultados = analisar_distribuicao_palavras(contador)


In [ ]:
def plotar_curva_com_corte(contador, corte_percentual=15, corte_palavras=35):
    contagens_ordenadas = np.array(sorted(contador.values(), reverse=True))
    soma_total = contagens_ordenadas.sum()
    cumulativa = np.cumsum(contagens_ordenadas) / soma_total

    plt.figure(figsize=(12,6))
    plt.plot(range(1, len(cumulativa)+1), cumulativa, marker='.', color='blue')
    plt.title('Curva de Frequência Cumulativa das Palavras')
    plt.xlabel('Top N palavras')
    plt.ylabel('Frequência acumulada')
    plt.grid(True)

    # Linha vertical no ponto de corte (35 palavras)
    plt.axvline(x=corte_palavras, linestyle='--', color='red', label=f'Corte: {corte_palavras} palavras')

    # Linha horizontal indicando o percentual acumulado (15%)
    plt.axhline(y=corte_percentual/100, linestyle='--', color='green', label=f'{corte_percentual}% acumulado')

    # Marcar ponto exato
    plt.scatter(corte_palavras, cumulativa[corte_palavras-1], color='red', zorder=5)
    plt.text(corte_palavras+20, cumulativa[corte_palavras-1]-0.02,
             f'{corte_palavras} palavras\n({corte_percentual}%)',
             color='red')

    plt.legend()
    plt.show()

plotar_curva_com_corte(contador, corte_percentual=20, corte_palavras=55)


## 4. Pré-Processamento

In [ ]:
# Executa pré-processamento
stopwords_extras = carregar_palavras_mais_frequentes(df['ementa'], stopwords_pt, top_n = TOP_N_WORDS)[0]
stopwords_pt.update(stopwords_extras)

df['ementa_preprocessada'] = df['ementa'].apply(lematizar_texto)
df['ementa_preprocessada'] = df['ementa_preprocessada'].apply(lambda x: remover_stopwords(x, stopwords_pt))

In [ ]:
df.head()

## 5. Análise Exploratória Depois do Pré-Processamento

### Recalcular estatísticas

In [ ]:
df['n_caracteres_preproc'] = df['ementa_preprocessada'].apply(contar_caracteres)
df['n_palavras_preproc'] = df['ementa_preprocessada'].apply(contar_palavras)

print(df[['n_palavras', 'n_palavras_preproc', 'n_caracteres', 'n_caracteres_preproc']].describe())




### Comparação antes e depois

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# Estatísticas
media_antes = df['n_palavras'].mean()
mediana_antes = df['n_palavras'].median()

media_depois = df['n_palavras_preproc'].mean()
mediana_depois = df['n_palavras_preproc'].median()

plt.figure(figsize=(12,7))

# Histograma Antes
sns.histplot(df['n_palavras'], bins=50, kde=True, color='royalblue', label='Antes do Pré-processamento', alpha=0.5)

# Histograma Depois
sns.histplot(df['n_palavras_preproc'], bins=50, kde=True, color='seagreen', label='Após o Pré-processamento', alpha=0.5)

# Linhas de Média
plt.axvline(media_antes, color='blue', linestyle='--', linewidth=2, label=f'Média Antes: {media_antes:.0f}')
plt.axvline(media_depois, color='green', linestyle='--', linewidth=2, label=f'Média Depois: {media_depois:.0f}')

# Linhas de Mediana
plt.axvline(mediana_antes, color='blue', linestyle='-', linewidth=2, label=f'Mediana Antes: {mediana_antes:.0f}')
plt.axvline(mediana_depois, color='green', linestyle='-', linewidth=2, label=f'Mediana Depois: {mediana_depois:.0f}')

plt.title('Distribuição do Número de Palavras nas Ementas\nAntes e Após o Pré-processamento', fontsize=14)
plt.xlabel('Número de Palavras por Ementa')
plt.ylabel('Frequência de Ementas')
plt.legend()
plt.grid(visible=True, linestyle='--', linewidth=0.5, alpha=0.7)

plt.tight_layout()
plt.show()


### Nuvem de palavras após pré-processamento

In [ ]:
# Junta cada lista de tokens em uma string
text_all_clean = ' '.join([' '.join(ementa) for ementa in df['ementa_preprocessada']])

# Gera a nuvem de palavras
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud_clean = WordCloud(
    width=800,
    height=400,
    background_color='white',
    colormap='plasma'
).generate(text_all_clean)

# Plota
plt.figure(figsize=(15,7))
plt.imshow(wordcloud_clean, interpolation='bilinear')
plt.axis('off')
plt.title('Nuvem de Palavras (Depois do Pré-processamento)', fontsize=16)
plt.show()


## Execução do modelo

In [ ]:
modelos = ['fasttext', 'doc2vec', 'sbert', 'bertimbau', 'legalbert']
df_resultados, embeddings_dict, labels_dict, reduzidos_dict, k_dict, modelos_cluster = comparar_modelos(
    modelos, df['ementa_preprocessada'], n_components=5,
    min_cluster_size=30
)

# Resultados

## Tabela de Comparação

In [ ]:
tabela = df_resultados.sort_values(by='Silhueta', ascending=False)

tabela_formatada = tabela.style.background_gradient(cmap='Reds', subset=['Tempo Pipeline (s)']) \
    .background_gradient(cmap='Blues_r', subset=['Silhueta']) \
    .background_gradient(cmap='Purples', subset=['Calinski-Harabasz']) \
    .background_gradient(cmap='Greens', subset=['Coherence c_v']) \
    .format(precision=4)

tabela_formatada


## Gráficos Comparativos

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(x=df_resultados.index, y='Silhueta', data=df_resultados.reset_index(), palette='viridis')
plt.title('Comparação da Métrica de Silhueta por Embedding', fontsize=14)
plt.ylabel('Coeficiente de Silhueta')
plt.xlabel('Modelo')
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.show()

plt.figure(figsize=(12,8))
sns.barplot(x=df_resultados.index, y='Calinski-Harabasz', data=df_resultados.reset_index(), palette='viridis')
plt.title('Comparação da Métrica de Calinski-Harabasz por Embedding', fontsize=14)
plt.ylabel('Calinski-Harabasz')
plt.xlabel('Modelo')
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.show()


## Radar Chart (Comparativo Multimétrico)

In [ ]:
from math import pi

def plot_radar(df):
    categorias = list(df.columns)
    N = len(categorias)

    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]  # Completa o círculo

    plt.figure(figsize=(8,8))
    ax = plt.subplot(111, polar=True)

    for idx, row in df.iterrows():
        valores = row.tolist()
        valores += valores[:1]
        ax.plot(angles, valores, linewidth=2, label=idx)
        ax.fill(angles, valores, alpha=0.1)

    plt.xticks(angles[:-1], categorias, color='grey', size=8)
    ax.set_rlabel_position(30)
    plt.title('Comparação Multimétrica dos Embeddings', size=14)
    plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1))
    plt.show()

# Seleciona as métricas que fazem sentido para radar (evita métricas com escalas muito discrepantes)
df_radar = df_resultados[['Silhueta', 'Calinski-Harabasz']].copy()

# Normaliza para radar (0-1)
for col in df_radar.columns:
    max_ = df_radar[col].max()
    min_ = df_radar[col].min()
    df_radar[col] = (df_radar[col] - min_) / (max_ - min_)

plot_radar(df_radar)


## Gerar Plot dos Clusters para Cada Modelo

In [ ]:
for modelo in modelos:
    plotar_clusters(
        reduzidos_dict[modelo],
        labels_dict[modelo],
        titulo=f'Clusters - {modelo.upper()} (K={k_dict[modelo]})'
    )



## Visualizar termos dos clusters

In [ ]:
for modelo in modelos:
    visualizar_clusters_termos(
        textos=df['ementa_preprocessada'],
        labels=labels_dict[modelo],
        modelo_nome=modelo,
        top_n=10,
        n_amostra=5
    )
